In [ ]:
import xarray as xr
from glob import glob
from os import path
import os
import numpy as np
import fsspec


# create NOAA OISST zarr data files



In [ ]:
%%time
adir = 'F:/data/sat_data/sst/noaa_oisst/www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/'
dir_pattern_zarr = adir + 'avhrr_zarr2/'
dir_pattern = adir + 'avhrr/'
pattern = dir_pattern + '/*/*.nc'
files = glob(pattern)
print('number of files:',len(files))

In [ ]:
%%time
#open dataset, this will take a while
ds=xr.open_mfdataset(files,combine='nested',concat_dim='time',decode_cf=False,mask_and_scale=False)
ds.close()
ds_all = ds.isel(zlev=0)
ds_all

In [ ]:
#remove any duplicates
_, index = np.unique(ds_all['time'], return_index=True)
ds_all=ds_all.isel(time=index)
#rechunck data  #data in int16 = 2 bytes 
ds_all = ds_all.chunk({'time':1000,'lat':300,'lon':300})
ds_all

In [ ]:
#output data to zarr format
#ds_all.to_zarr(dir_pattern_zarr, consolidated=True)

In [ ]:
ds_all

# append new data to original data store without overwritting entire store

- there is a trick, I need to raise and issue on github to document because the documentation isn't clear about:
1. that you are appending, so you need to make sure you are only appending the new bit
2. that you need to decode your data

I struggled to get this to work at first because I tried just reading the data the same way & then appending it. 
This did not work, I got errors about removing scale_factor, add_offset, Fill_value which are all encoding attributes. After poking around, the error suggest you just remove these attributes, and that is true, if you remove them, it will work, but then the appended data isn't encoded correctly and it is a mess.  so, actually, you have to apply the decoding, then everything is peachy. 

## First calculate where the old data ends, then read in new data, decode, append

In [ ]:
ds_old = xr.open_zarr(dir_pattern_zarr, consolidated=True,decode_cf=False)
ds_old.close()
lasttime=ds_old.time[-1] + 1#+np.timedelta64(1, 'D')
print(lasttime.data)
ds_old

In [ ]:
ds=xr.open_mfdataset(files[-100:],combine='nested',concat_dim='time',decode_cf=False,mask_and_scale=False)
ds.close()
ds_all = ds.isel(zlev=0)
#remove any duplicates
_, index = np.unique(ds_all['time'], return_index=True)
ds_all=ds_all.isel(time=index)
#rechunck data  #data in int16 = 2 bytes 
ds_all = ds_all.chunk({'time':1000,'lat':300,'lon':300})
ds_new = ds_all.sel(time=slice(lasttime,9999999))
print(ds_new.time[0:3].data)
ds_new = xr.decode_cf(ds_new)
ds_new.to_zarr(dir_pattern_zarr, mode='a',append_dim='time', consolidated=True) #


# slightly different version for AWS Hub

In [ ]:
ikey = fsspec.get_mapper('s3://noaa-oisstv02r2/', anon=False)
ds_old = xr.open_zarr(ikey,consolidated=True, decode_cf=False)
ds_old.close()
lasttime=ds_old.time[-1] + 1#+np.timedelta64(1, 'D')
print(lasttime.data)
ds_old

In [ ]:
xr.decode_cf(ds_old.time[-1],)

In [ ]:
import s3fs
import xarray as xr

s3 = s3fs.S3FileSystem(anon=False)
# This generates a list of strings with filenames
s3path = 's3://noaa-oisstv02r1/*/*.nc'
remote_files = s3.glob(s3path)

# Iterate through remote_files to create a fileset
fileset = [s3.open(file) for file in remote_files]


In [ ]:
ds=xr.open_mfdataset(fileset,combine='nested',concat_dim='time',decode_cf=False,mask_and_scale=False)
ds.close()
ds_all = ds.isel(zlev=0)
ds_all

In [ ]:
#remove any duplicates
_, index = np.unique(ds_all['time'], return_index=True)
ds_all=ds_all.isel(time=index)
#rechunck data  #data in int16 = 2 bytes 
ds_all = ds_all.chunk({'time':1000,'lat':300,'lon':300})
ds_new = ds_all.sel(time=slice(lasttime,9999999))
print(ds_new.time[0:3].data)
ds_new = xr.decode_cf(ds_new)
ikey = fsspec.get_mapper('s3://noaa-oisstv02r2/', anon=False)
ds_new.to_zarr(ikey, mode='a',append_dim='time', consolidated=True) 